In [ ]:
import pygame

In [ ]:
GRID_SIZE = 3

In [ ]:
class PygameInit:

    @classmethod
    def initialization(cls):
        grid_size_x = GRID_SIZE
        grid_size_y = GRID_SIZE
        tile_size = 200

        pygame.init()
        screen = pygame.display.set_mode((grid_size_x * tile_size, grid_size_y * tile_size))
        pygame.display.set_caption("FOL")
        clock = pygame.time.Clock()

        return screen, clock